# Mushroom Classification Project 🍄

## Akbank Makine Öğrenmesi Bootcamp - Gözetimli Öğrenme Projesi

Bu projede, Kaggle'dan alınan "Mushroom Classification" veri setini kullanarak mantarların yenilebilir (edible) veya zehirli (poisonous) olup olmadığını tahmin eden bir makine öğrenmesi modeli geliştireceğiz.

### Proje Hedefleri:
- Kapsamlı Keşifsel Veri Analizi (EDA) gerçekleştirmek
- Veri ön işleme tekniklerini uygulamak
- Çoklu algoritma karşılaştırması yapmak
- En iyi performans gösteren modeli belirlemek

### Kullanılacak Algoritmalar:
- Logistic Regression
- Ridge Classifier
- Decision Tree
- Gaussian Naive Bayes
- Neural Networks (MLP)
- Random Forest

---

## 1. Kütüphanelerin İçe Aktarılması

Projemizde kullanacağımız tüm kütüphaneleri içe aktarıyoruz.

In [ ]:
# Import Pandas and Matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Import Label Encoder and train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [ ]:
# Import Logistic Regression, Ridge Classifier, Decision Tree
# Gaussian Naive Bayes, MLP Classifier and Random Forest models
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Import Classification Report function
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

## 2. Veri Seti ve Ön İşleme

Veri setimiz 8124 mantar örneği içermektedir. Her mantar örneği 22 özelliğe sahiptir ve yenilebilir veya zehirli olarak kategorize edilmiştir.

### 2.1 Veriyi Okuma

In [ ]:
# Read the "mushroom.csv" file
data = pd.read_csv('mushrooms.csv')

print(f"Veri seti boyutu: {data.shape}")
print(f"Toplam veri noktası: {len(data)}")
print(f"Özellik sayısı: {data.shape[1] - 1}")

### 2.2 Veriyi Görselleştirme

In [ ]:
# Use the head() function to display the first 5 rows of the data
print("İlk 5 satır:")
data.head()

In [ ]:
# Veri seti hakkında genel bilgiler
print("Veri seti bilgileri:")
print(data.info())
print("\n" + "="*50 + "\n")
print("İstatistiksel özet:")
print(data.describe())

In [ ]:
# Eksik değerleri kontrol etme
print("Eksik değerler:")
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])

# '?' karakterini eksik değer olarak kontrol etme
print("\n'?' karakteri içeren sütunlar:")
for col in data.columns:
    question_marks = (data[col] == '?').sum()
    if question_marks > 0:
        print(f"{col}: {question_marks} adet '?' değeri")

## 3. Keşifsel Veri Analizi (EDA)

Veri setimizi daha iyi anlamak için kapsamlı bir analiz gerçekleştireceğiz.

In [ ]:
# Use value_counts method on "class" column of data object
class_counts = data['class'].value_counts()

# Print the result
print("Sınıf dağılımı:")
print(class_counts)
print(f"\nYüzdelik dağılım:")
print(data['class'].value_counts(normalize=True) * 100)

In [ ]:
# Add the bar for edible class
# Add the bar for poisonous class
# Print the plot

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
class_counts.plot(kind='bar', color=['#2E8B57', '#DC143C'])
plt.title('Mantar Sınıflarının Dağılımı', fontsize=14, fontweight='bold')
plt.xlabel('Sınıf (e: Yenilebilir, p: Zehirli)', fontsize=12)
plt.ylabel('Sayı', fontsize=12)
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
plt.pie(class_counts.values, labels=['Yenilebilir', 'Zehirli'], 
        colors=['#2E8B57', '#DC143C'], autopct='%1.1f%%', startangle=90)
plt.title('Sınıf Dağılımı (Yüzde)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Kategorik değişkenlerin benzersiz değer sayıları
print("Her sütundaki benzersiz değer sayıları:")
for col in data.columns:
    unique_count = data[col].nunique()
    print(f"{col}: {unique_count} benzersiz değer")

In [ ]:
# En önemli özelliklerin görselleştirilmesi
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Önemli Özelliklerin Sınıflara Göre Dağılımı', fontsize=16, fontweight='bold')

important_features = ['odor', 'gill-size', 'gill-color', 'stalk-shape', 'cap-color', 'bruises']

for i, feature in enumerate(important_features):
    row = i // 3
    col = i % 3
    
    cross_tab = pd.crosstab(data[feature], data['class'])
    cross_tab.plot(kind='bar', ax=axes[row, col], color=['#2E8B57', '#DC143C'])
    axes[row, col].set_title(f'{feature.title()} vs Sınıf', fontweight='bold')
    axes[row, col].set_xlabel(feature.title())
    axes[row, col].set_ylabel('Sayı')
    axes[row, col].legend(['Yenilebilir', 'Zehirli'])
    axes[row, col].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Veri Ön İşleme

Makine öğrenmesi algoritmalarının kategorik verileri işleyebilmesi için Label Encoding uygulayacağız.

In [ ]:
# Create the X variable for features
X = data.drop('class', axis=1)

# Create the y variable for output labels
y = data['class']

print(f"Özellik matrisi boyutu: {X.shape}")
print(f"Hedef değişken boyutu: {y.shape}")

In [ ]:
# Create an LabelEncoder object
label_encoder = LabelEncoder()

# Encode the features to integers inside a for loop
for column in X.columns:
    X[column] = label_encoder.fit_transform(X[column])

# Encode the output labels to integers
y = label_encoder.fit_transform(y)

print("Label Encoding tamamlandı!")
print(f"Hedef değişken: 0 = Yenilebilir, 1 = Zehirli")

In [ ]:
# Print X
print("Kodlanmış özellik matrisi (ilk 5 satır):")
print(X.head())

In [ ]:
# Print y
print("Kodlanmış hedef değişken (ilk 10 değer):")
print(y[:10])
print(f"\nHedef değişken dağılımı:")
unique, counts = np.unique(y, return_counts=True)
for i, (val, count) in enumerate(zip(unique, counts)):
    label = "Yenilebilir" if val == 0 else "Zehirli"
    print(f"{val} ({label}): {count} örnek")

## 5. Veri Setini Bölme

Veri setimizi %70 eğitim, %30 test olacak şekilde böleceğiz.

In [ ]:
# Split the dataset into train and test sets with 70-30 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")
print(f"Eğitim seti hedef dağılımı: {np.bincount(y_train)}")
print(f"Test seti hedef dağılımı: {np.bincount(y_test)}")

## 6. Model Oluşturma

Farklı makine öğrenmesi algoritmalarını kullanarak modeller oluşturacağız.

In [ ]:
# Create an object using the LogisticRegression() class
logistic_model = LogisticRegression(random_state=42, max_iter=1000)

# Create an object using the RidgeClassifier() class
ridge_model = RidgeClassifier(random_state=42)

# Create an object using the DecisionTreeClassifier() class
decision_tree_model = DecisionTreeClassifier(random_state=42)

# Create an object using the GaussianNB() class
naive_bayes_model = GaussianNB()

# Create an object using the MLPClassifier() class
neural_network_model = MLPClassifier(random_state=42, max_iter=1000)

print("Tüm modeller oluşturuldu!")

## 7. Model Eğitimi

Oluşturduğumuz modelleri eğitim verisi ile eğiteceğiz.

In [ ]:
# Train the Logistic Classifier model
logistic_model.fit(X_train, y_train)
print("Logistic Regression modeli eğitildi ✓")

# Train the Ridge Classifier model
ridge_model.fit(X_train, y_train)
print("Ridge Classifier modeli eğitildi ✓")

# Train the Decision Tree model
decision_tree_model.fit(X_train, y_train)
print("Decision Tree modeli eğitildi ✓")

# Train the Naive Bayes model
naive_bayes_model.fit(X_train, y_train)
print("Naive Bayes modeli eğitildi ✓")

# Train the Neural Network model
neural_network_model.fit(X_train, y_train)
print("Neural Network modeli eğitildi ✓")

## 8. Tahmin Yapma

Eğitilmiş modellerimizi kullanarak test verisi üzerinde tahminler yapacağız.

In [ ]:
# Make prediction using the test dataset on Logistic Classifier model
logistic_predictions = logistic_model.predict(X_test)

# Make prediction using the test dataset on Ridge Classifier model
ridge_predictions = ridge_model.predict(X_test)

# Make prediction using the test dataset on Decision Tree model
decision_tree_predictions = decision_tree_model.predict(X_test)

# Make prediction using the test dataset on Naive Bayes model
naive_bayes_predictions = naive_bayes_model.predict(X_test)

# Make prediction using the test dataset on Neural Network model
neural_network_predictions = neural_network_model.predict(X_test)

print("Tüm modeller için tahminler tamamlandı!")

## 9. Sınıflandırma Raporları Oluşturma

Her model için detaylı performans raporları oluşturacağız.

In [ ]:
# Create a Classification Report for Logistic Classifier model
logistic_report = classification_report(y_test, logistic_predictions, 
                                       target_names=['Yenilebilir', 'Zehirli'])

# Create a Classification Report for Ridge Classifier model
ridge_report = classification_report(y_test, ridge_predictions, 
                                   target_names=['Yenilebilir', 'Zehirli'])

# Create a Classification Report for Decision Tree model
decision_tree_report = classification_report(y_test, decision_tree_predictions, 
                                            target_names=['Yenilebilir', 'Zehirli'])

# Create a Classification Report for Naive Bayes model
naive_bayes_report = classification_report(y_test, naive_bayes_predictions, 
                                         target_names=['Yenilebilir', 'Zehirli'])

# Create a Classification Report for Neural Network model
neural_network_report = classification_report(y_test, neural_network_predictions, 
                                             target_names=['Yenilebilir', 'Zehirli'])

print("Tüm sınıflandırma raporları oluşturuldu!")

## 10. Model Performanslarının Görüntülenmesi

Her modelin performansını detaylı olarak inceleyeceğiz.

In [ ]:
# Print the report of the Logistic Regression model
print("=" * 60)
print("LOGISTIC REGRESSION MODEL PERFORMANSI")
print("=" * 60)
print(logistic_report)
print(f"Doğruluk Oranı: {accuracy_score(y_test, logistic_predictions):.4f}")

In [ ]:
# Print the report of the Ridge Regression model
print("=" * 60)
print("RIDGE CLASSIFIER MODEL PERFORMANSI")
print("=" * 60)
print(ridge_report)
print(f"Doğruluk Oranı: {accuracy_score(y_test, ridge_predictions):.4f}")

In [ ]:
# Print the report of the Decision Tree model
print("=" * 60)
print("DECISION TREE MODEL PERFORMANSI")
print("=" * 60)
print(decision_tree_report)
print(f"Doğruluk Oranı: {accuracy_score(y_test, decision_tree_predictions):.4f}")

In [ ]:
# Print the report of the Naive Bayes model
print("=" * 60)
print("NAIVE BAYES MODEL PERFORMANSI")
print("=" * 60)
print(naive_bayes_report)
print(f"Doğruluk Oranı: {accuracy_score(y_test, naive_bayes_predictions):.4f}")

In [ ]:
# Print the report of the Neural Network model
print("=" * 60)
print("NEURAL NETWORK MODEL PERFORMANSI")
print("=" * 60)
print(neural_network_report)
print(f"Doğruluk Oranı: {accuracy_score(y_test, neural_network_predictions):.4f}")

## 11. Random Forest Modeli

Ek olarak Random Forest algoritmasını da test edeceğiz.

In [ ]:
# Create Random Forest Classifier object, train it and make predictions
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)
random_forest_predictions = random_forest_model.predict(X_test)

print("Random Forest modeli eğitildi ve tahminler yapıldı!")

In [ ]:
# Create a classification Report for Random Forest model
random_forest_report = classification_report(y_test, random_forest_predictions, 
                                            target_names=['Yenilebilir', 'Zehirli'])

# Print the classification report
print("=" * 60)
print("RANDOM FOREST MODEL PERFORMANSI")
print("=" * 60)
print(random_forest_report)
print(f"Doğruluk Oranı: {accuracy_score(y_test, random_forest_predictions):.4f}")

## 12. Model Karşılaştırması ve Sonuçlar

Tüm modellerin performansını karşılaştırarak en iyi modeli belirleyeceğiz.

In [ ]:
# Model performanslarını karşılaştırma
models = {
    'Logistic Regression': logistic_predictions,
    'Ridge Classifier': ridge_predictions,
    'Decision Tree': decision_tree_predictions,
    'Naive Bayes': naive_bayes_predictions,
    'Neural Network': neural_network_predictions,
    'Random Forest': random_forest_predictions
}

results = []
for name, predictions in models.items():
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Accuracy', ascending=False)

print("=" * 80)
print("TÜM MODELLERİN PERFORMANS KARŞILAŞTIRMASI")
print("=" * 80)
print(results_df.round(4))

In [ ]:
# Performans görselleştirmesi
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Model Performans Karşılaştırması', fontsize=16, fontweight='bold')

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for i, metric in enumerate(metrics):
    row = i // 2
    col = i % 2
    
    ax = axes[row, col]
    bars = ax.bar(results_df['Model'], results_df[metric], color=colors[i], alpha=0.7)
    ax.set_title(f'{metric} Karşılaştırması', fontweight='bold')
    ax.set_ylabel(metric)
    ax.tick_params(axis='x', rotation=45)
    
    # Değerleri çubukların üzerine yazma
    for bar, value in zip(bars, results_df[metric]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    ax.set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

In [ ]:
# En iyi modeli belirleme
best_model = results_df.iloc[0]
print("=" * 60)
print("EN İYİ PERFORMANS GÖSTEREN MODEL")
print("=" * 60)
print(f"Model: {best_model['Model']}")
print(f"Doğruluk: {best_model['Accuracy']:.4f}")
print(f"Kesinlik: {best_model['Precision']:.4f}")
print(f"Duyarlılık: {best_model['Recall']:.4f}")
print(f"F1-Skor: {best_model['F1-Score']:.4f}")

## 13. Confusion Matrix Analizi

En iyi performans gösteren modeller için confusion matrix analizi yapacağız.

In [ ]:
# En iyi 3 modelin confusion matrix'lerini görselleştirme
top_3_models = results_df.head(3)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('En İyi 3 Modelin Confusion Matrix Analizi', fontsize=16, fontweight='bold')

for i, (_, row) in enumerate(top_3_models.iterrows()):
    model_name = row['Model']
    predictions = models[model_name]
    
    cm = confusion_matrix(y_test, predictions)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Yenilebilir', 'Zehirli'],
                yticklabels=['Yenilebilir', 'Zehirli'],
                ax=axes[i])
    
    axes[i].set_title(f'{model_name}\nAccuracy: {row["Accuracy"]:.4f}', fontweight='bold')
    axes[i].set_xlabel('Tahmin Edilen')
    axes[i].set_ylabel('Gerçek')

plt.tight_layout()
plt.show()

## 14. Özellik Önem Analizi

Random Forest modelini kullanarak hangi özelliklerin en önemli olduğunu analiz edeceğiz.

In [ ]:
# Özellik önem analizi
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': random_forest_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
plt.barh(range(len(feature_importance)), feature_importance['importance'])
plt.yticks(range(len(feature_importance)), feature_importance['feature'])
plt.xlabel('Önem Skoru')
plt.title('Random Forest - Özellik Önem Analizi', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()

# En önemli 10 özelliği vurgulama
for i in range(min(10, len(feature_importance))):
    plt.barh(i, feature_importance.iloc[i]['importance'], color='red', alpha=0.7)

plt.tight_layout()
plt.show()

print("En önemli 10 özellik:")
print(feature_importance.head(10))

## 15. Cross-Validation Analizi

Modellerimizin güvenilirliğini test etmek için cross-validation analizi yapacağız.

In [ ]:
# Cross-validation analizi
model_objects = {
    'Logistic Regression': logistic_model,
    'Ridge Classifier': ridge_model,
    'Decision Tree': decision_tree_model,
    'Naive Bayes': naive_bayes_model,
    'Neural Network': neural_network_model,
    'Random Forest': random_forest_model
}

cv_results = []
for name, model in model_objects.items():
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    cv_results.append({
        'Model': name,
        'CV_Mean': cv_scores.mean(),
        'CV_Std': cv_scores.std(),
        'CV_Scores': cv_scores
    })

cv_df = pd.DataFrame(cv_results)
cv_df = cv_df.sort_values('CV_Mean', ascending=False)

print("=" * 70)
print("5-FOLD CROSS-VALIDATION SONUÇLARI")
print("=" * 70)
for _, row in cv_df.iterrows():
    print(f"{row['Model']:20} | Ortalama: {row['CV_Mean']:.4f} ± {row['CV_Std']:.4f}")

In [ ]:
# Cross-validation sonuçlarını görselleştirme
plt.figure(figsize=(12, 8))
cv_scores_list = [row['CV_Scores'] for _, row in cv_df.iterrows()]
model_names = cv_df['Model'].tolist()

plt.boxplot(cv_scores_list, labels=model_names)
plt.title('Cross-Validation Skor Dağılımları', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy Score')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 16. Sonuç ve Değerlendirme

### Proje Özeti

Bu projede, mantar sınıflandırması problemi için 6 farklı makine öğrenmesi algoritması karşılaştırılmıştır:

1. **Logistic Regression**
2. **Ridge Classifier** 
3. **Decision Tree**
4. **Gaussian Naive Bayes**
5. **Neural Network (MLP)**
6. **Random Forest**

### Ana Bulgular:

- **Veri Seti**: 8124 mantar örneği, 22 özellik
- **Sınıf Dağılımı**: Dengeli dağılım (yaklaşık %50-%50)
- **En İyi Model**: Analiz sonucunda belirlenen en yüksek performanslı model
- **Kritik Özellikler**: Özellik önem analizi ile belirlenen en etkili faktörler

### Gerçek Hayat Uygulamaları:

1. **Doğa Yürüyüşü Uygulamaları**: Mobil uygulamalarda mantar tanıma
2. **Eğitim Araçları**: Biyoloji eğitiminde kullanım
3. **Güvenlik Sistemleri**: Yiyecek güvenliği kontrolü
4. **Araştırma**: Mikrobiyoloji ve botanik araştırmaları

### Gelecek Geliştirmeler:

1. **Görüntü İşleme**: Mantar fotoğraflarından otomatik özellik çıkarımı
2. **Ensemble Yöntemler**: Birden fazla modelin kombinasyonu
3. **Deep Learning**: CNN tabanlı görüntü sınıflandırması
4. **Mobil Uygulama**: Gerçek zamanlı mantar tanıma uygulaması
5. **Veri Artırma**: Daha fazla mantar türü ve özellik eklenmesi

### Teknik Başarılar:

- Yüksek doğruluk oranları elde edildi
- Cross-validation ile model güvenilirliği doğrulandı
- Özellik önem analizi ile kritik faktörler belirlendi
- Kapsamlı model karşılaştırması gerçekleştirildi

Bu proje, makine öğrenmesi tekniklerinin biyolojik sınıflandırma problemlerindeki etkinliğini göstermektedir.